In [1]:
# =========================
# 0. Paths & Imports
# =========================
import os, io, json, random, pickle, gc
from pathlib import Path
import pandas as pd
import numpy as np

from PIL import Image
from sklearn.model_selection import train_test_split

# Try multilabel stratification; fallback to random split if unavailable
try:
    from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
    HAS_ITERSTRAT = True
except Exception:
    HAS_ITERSTRAT = False

BASE = "/scratch/jen.che/nutrition5k_cache/datasets/siddhantrout/nutrition5k-dataset"

EXCEL_DISH_INGR = f"{BASE}/dish_ingredients.xlsx"
EXCEL_DISHES = f"{BASE}/dishes.xlsx"
EXCEL_INGRS = f"{BASE}/ingredients.xlsx"
PKL_IMAGES = f"{BASE}/dish_images.pkl"   # byte-encoded rgb/depth

OUT_ROOT = Path("/scratch/jen.che/nutrition5k_prepared")
IMG_DIR = OUT_ROOT / "images"
OUT_ROOT.mkdir(parents=True, exist_ok=True)
IMG_DIR.mkdir(parents=True, exist_ok=True)

print("Output root:", OUT_ROOT)
print("Images dir:", IMG_DIR)


Output root: /scratch/jen.che/nutrition5k_prepared
Images dir: /scratch/jen.che/nutrition5k_prepared/images


In [2]:
# =========================
# 1. Load tables
# =========================
dish_ingr = pd.read_excel(EXCEL_DISH_INGR)
dishes = pd.read_excel(EXCEL_DISHES)
ingrs = pd.read_excel(EXCEL_INGRS)

print(dish_ingr.shape, dishes.shape, ingrs.shape)
display(dish_ingr.head())
display(dishes.head())
display(ingrs.head())

# Normalize column names across sources
# Siddhant’s dump uses columns like:
# dish_ingredients: [dish_id, ingr_id, ingr_name, grams, calories, fat, carb, protein]
# ingredients:      [ingr, id, cal/g, fat(g), carb(g), protein(g)]
# dishes:           [dish_id, total_mass, total_calories, total_fat, total_carb, total_protein]

dish_ingr = dish_ingr.rename(columns=str.strip)
ingrs = ingrs.rename(columns={"ingr":"name"})  # keep a 'name' column


(28455, 8) (5006, 6) (555, 6)


,dish_id,ingr_id,ingr_name,grams,calories,fat,carb,protein
0,dish_1561662216,ingr_0000000508,soy sauce,3.398568,1.801241,0.020391,0.166530,0.275284
1,dish_1561662216,ingr_0000000122,garlic,2.124105,3.164916,0.010621,0.700955,0.135943
2,dish_1561662216,ingr_0000000026,white rice,8.496420,11.045346,0.025489,2.378998,0.229403
3,dish_1561662216,ingr_0000000524,parsley,0.213397,0.079170,0.001707,0.013657,0.006189
4,dish_1561662216,ingr_0000000094,onions,1.707173,0.682869,0.001707,0.153646,0.018779


,dish_id,total_mass,total_calories,total_fat,total_carb,total_protein
0,dish_1561662216,300.794281,193,12.387489,28.218290,18.633970
1,dish_1562688426,137.569992,88,8.256000,5.190000,10.297000
2,dish_1561662054,419.438782,292,23.838249,26.351543,25.910593
3,dish_1562008979,382.936646,290,22.224644,10.173570,35.345387
4,dish_1560455030,20.590000,103,0.148000,4.625000,0.956000


,ingr,id,cal/g,fat(g),carb(g),protein(g)
0,cottage cheese,1,0.980,0.043,0.034,0.110
1,strawberries,2,0.330,0.003,0.080,0.007
2,garden salad,3,0.646,0.034,0.032,0.061
3,bacon,4,5.410,0.420,0.014,0.370
4,potatoes,5,0.770,0.001,0.170,0.020


In [3]:
# =========================
# 2. Build ingredient vocabulary
# =========================
# Option A: full vocab (all ingredients)
vocab = sorted(dish_ingr["ingr_name"].dropna().unique().tolist())

# Option B: top-K frequent classes (more stable for a first baseline)
# freq = dish_ingr["ingr_name"].value_counts()
# K = 150
# vocab = sorted(freq.head(K).index.tolist())

ingr2idx = {name:i for i, name in enumerate(vocab)}
idx2ingr = {i:name for name,i in ingr2idx.items()}
print("Vocab size:", len(vocab))
print("Sample:", vocab[:10])

# Save vocab
with open(OUT_ROOT / "ingredient_vocab.json", "w") as f:
    json.dump({"vocab": vocab}, f, indent=2)


Vocab size: 249
Sample: ['almond milk', 'almonds', 'apple', 'apple cider', 'artichokes', 'arugula', 'asparagus', 'avocado', 'baby carrots', 'bacon']


In [4]:
# =========================
# 3. Prepare multilabel targets per dish
# =========================
# Aggregate ingredients for each dish into a multi-hot vector over vocab
def to_multi_hot(names):
    vec = np.zeros(len(vocab), dtype=np.uint8)
    for n in names:
        if n in ingr2idx:
            vec[ingr2idx[n]] = 1
    return vec

dish_to_ingrs = (dish_ingr.groupby("dish_id")["ingr_name"]
                 .apply(lambda s: sorted(set(s.dropna().tolist())))
                 .reset_index(name="ingredients"))

dish_to_ingrs["label_vec"] = dish_to_ingrs["ingredients"].apply(to_multi_hot)
dish_to_ingrs["num_labels"] = dish_to_ingrs["label_vec"].apply(lambda v: int(v.sum()))
dish_to_ingrs = dish_to_ingrs[dish_to_ingrs["num_labels"]>0].reset_index(drop=True)

print("Dishes retained:", len(dish_to_ingrs))
display(dish_to_ingrs.head(3))


Dishes retained: 5006


,dish_id,ingredients,label_vec,num_labels
0,dish_1550704750,[chicken],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
1,dish_1550704903,[bell peppers],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
2,dish_1550705288,[potatoes],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1


In [16]:
import os, pickle, io
import pandas as pd
from PIL import Image
from tqdm import tqdm

# === 基本設定 ===
dataset_dir = "/scratch/jen.che/nutrition5k_cache/datasets/siddhantrout/nutrition5k-dataset"
prepared_dir = "/scratch/jen.che/nutrition5k_prepared"
os.makedirs(prepared_dir, exist_ok=True)

pkl_path = os.path.join(dataset_dir, "dish_images.pkl")
output_dir = os.path.join(prepared_dir, "images")
os.makedirs(output_dir, exist_ok=True)

manifest_path = os.path.join(prepared_dir, "manifest.csv")

# === 如果 manifest 已存在，直接載入，不重複解碼 ===
if os.path.exists(manifest_path):
    print(f"Manifest already exists → {manifest_path}")
    manifest_df = pd.read_csv(manifest_path)
    print("✅ Loaded manifest with shape:", manifest_df.shape)
    print(manifest_df.head())
else:
    # === 讀取 pickle ===
    with open(pkl_path, "rb") as f:
        data = pickle.load(f)

    print("Type of loaded object:", type(data))
    print("Columns:", list(data.columns))

    # 若是 DataFrame，就轉成 records (list of dicts)
    if isinstance(data, pd.DataFrame):
        rows = data.to_dict(orient="records")
    else:
        rows = data

    print(f"Loaded {len(rows)} entries")
    print("Sample keys:", list(rows[0].keys()))

    # === 檢查欄位 ===
    id_key = "dish_id" if "dish_id" in rows[0] else "dish"
    rgb_key = "rgb_image" if "rgb_image" in rows[0] else "rgb"
    print(f"Using id_key='{id_key}', rgb_key='{rgb_key}'")

    # === 讀取 dish_ingredients 以取得有效 dish_id ===
    dish_ingr = pd.read_excel(os.path.join(dataset_dir, "dish_ingredients.xlsx"))
    valid_dishes = set(dish_ingr["dish_id"])

    saved, skipped = 0, 0
    manifest = []

    for row in tqdm(rows, desc="Decoding images"):
        dish_id = row[id_key]
        rgb_bytes = row[rgb_key]

        # 僅保留有 ingredient 的 dish
        if dish_id not in valid_dishes:
            continue

        try:
            img = Image.open(io.BytesIO(rgb_bytes)).convert("RGB")
            out_path = os.path.join(output_dir, f"{dish_id}.jpg")
            img.save(out_path, format="JPEG")
            manifest.append([dish_id, out_path])
            saved += 1
        except Exception as e:
            skipped += 1

    print(f"✅ Saved {saved} images, skipped {skipped}")

    # === 儲存 manifest ===
    manifest_df = pd.DataFrame(manifest, columns=["dish_id", "image_path"])
    manifest_df.to_csv(manifest_path, index=False)
    print(f"✅ Manifest saved → {manifest_path}")
    print(manifest_df.head())


Type of loaded object: <class 'pandas.core.frame.DataFrame'>
Columns: ['dish', 'rgb_image', 'depth_image']
Loaded 3490 entries
Sample keys: ['dish', 'rgb_image', 'depth_image']
Using id_key='dish', rgb_key='rgb_image'


Decoding images: 100%|██████████| 3490/3490 [01:17<00:00, 44.91it/s]

✅ Saved 3490 images, skipped 0
✅ Manifest saved → /scratch/jen.che/nutrition5k_prepared/manifest.csv
           dish_id                                         image_path
0  dish_1559243887  /scratch/jen.che/nutrition5k_prepared/images/d...
1  dish_1563568319  /scratch/jen.che/nutrition5k_prepared/images/d...
2  dish_1558641007  /scratch/jen.che/nutrition5k_prepared/images/d...
3  dish_1558109945  /scratch/jen.che/nutrition5k_prepared/images/d...
4  dish_1563216412  /scratch/jen.che/nutrition5k_prepared/images/d...


Step 5

In [22]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
import os

# === 路徑設定 ===
dataset_dir = "/scratch/jen.che/nutrition5k_cache/datasets/siddhantrout/nutrition5k-dataset"
prepared_dir = "/scratch/jen.che/nutrition5k_prepared"
os.makedirs(prepared_dir, exist_ok=True)

# === 載入資料 ===
dish_ingr_path = os.path.join(dataset_dir, "dish_ingredients.xlsx")
manifest_path = os.path.join(prepared_dir, "manifest.csv")

dish_ingr = pd.read_excel(dish_ingr_path)

# 安全載入 manifest_df
if os.path.exists(manifest_path):
    manifest_df = pd.read_csv(manifest_path)
    print(f"成功載入 manifest.csv，共 {len(manifest_df)} 筆資料")
else:
    raise FileNotFoundError("找不到 manifest.csv，請先完成影像解碼步驟（Step 4）")

# === 建立食材字典 ===
dish_ingr["ingr_name"] = dish_ingr["ingr_name"].astype(str).str.strip()
dish_ingr = dish_ingr[dish_ingr["ingr_name"].notna()]
dish_ingr = dish_ingr[dish_ingr["ingr_name"] != "nan"]

ingredient_vocab = sorted(dish_ingr["ingr_name"].unique().tolist())
ingredient_to_idx = {name: i for i, name in enumerate(ingredient_vocab)}
print(f"共有 {len(ingredient_vocab)} 種食材")

# === 為每道菜建立標籤向量 ===
records = []
for dish_id, group in dish_ingr.groupby("dish_id"):
    label_vec = [0] * len(ingredient_vocab)
    ingr_list = group["ingr_name"].tolist()
    for ingr in ingr_list:
        idx = ingredient_to_idx[ingr]
        label_vec[idx] = 1
    records.append({
        "dish_id": dish_id,
        "ingredients": ingr_list,
        "label_json": json.dumps(label_vec)
    })

labels_df = pd.DataFrame(records)
print("標籤表格預覽：")
print(labels_df.head())

# === 確保欄位名稱一致 ===
for df_name, df in [("manifest_df", manifest_df), ("labels_df", labels_df)]:
    if "dish" in df.columns and "dish_id" not in df.columns:
        df.rename(columns={"dish": "dish_id"}, inplace=True)

# === 安全檢查欄位 ===
if "dish_id" not in manifest_df.columns:
    raise KeyError("manifest_df 缺少 dish_id 欄位")
if "dish_id" not in labels_df.columns:
    raise KeyError("labels_df 缺少 dish_id 欄位")

# === 合併影像與標籤 ===
merged = pd.merge(manifest_df, labels_df, on="dish_id", how="inner")
print(f"合併成功，共 {len(merged)} 筆資料")
print(merged.head())

# === 分割 train / val / test ===
train_df, test_df = train_test_split(merged, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(test_df, test_size=0.5, random_state=42)

# === 儲存 ===
for name, df in [("train", train_df), ("val", val_df), ("test", test_df)]:
    path = os.path.join(prepared_dir, f"{name}.csv")
    df.to_csv(path, index=False)
    print(f"Saved {len(df)} → {path}")

print("全部完成！train/val/test CSV 已建立。")


✅ 成功載入 manifest.csv，共 3490 筆資料
共有 249 種食材
標籤表格預覽：
           dish_id     ingredients  \
0  dish_1550704750       [chicken]   
1  dish_1550704903  [bell peppers]   
2  dish_1550705288      [potatoes]   
3  dish_1550705370          [beef]   
4  dish_1550705445     [cucumbers]   

                                          label_json  
0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
2  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
3  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
4  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
✅ 合併成功，共 3490 筆資料
           dish_id                                         image_path  \
0  dish_1559243887  /scratch/jen.che/nutrition5k_prepared/images/d...   
1  dish_1563568319  /scratch/jen.che/nutrition5k_prepared/images/d...   
2  dish_1558641007  /scratch/jen.che/nutrition5k_prepared/images/d...   
3  dish_1558109945  /scratch/jen.che/nutrition5k_prepared/images/d...   
4  dish_